<a href="https://colab.research.google.com/github/luizahenriques/perfilpoliticos/blob/main/2024_01_19Extra%C3%A7%C3%A3oNot%C3%ADciasG1SpicyIncremental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install spacy
!python -m spacy download pt_core_news_sm


In [ ]:
%%capture
!pip install google-auth gspread

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from google.colab import drive
from datetime import datetime
import os
import spacy
import shutil
import gspread
from google.colab import auth



In [ ]:
import numpy as np
import pandas as pd
#For displaying complete rows info
pd.options.display.max_colwidth=500
import tensorflow as tf
import spacy
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from collections import Counter

Salvar arquivos de notícias diários em uma pasta do Google Drive

In [ ]:
#Pasta na qual serão armazenados Fonte Original os arquivos do G1 diários
pasta_drive_noticias = '/content/drive/My Drive/PesquisaPoliticos/Noticias/G1/Fonte_Original/'

In [ ]:
#Pasta em que cada arquivo de Deputado (com notícias relacionadas  -Score>4) será salvo
pasta_drive_noticias_deputados = '/content/drive/My Drive/PesquisaPoliticos/Noticias/G1/Noticias_Deputados/'

In [ ]:
# Montar o Google Drive no Colab
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Obter a data atual para compor o nome do arquivo no Google Drive
data_atual = datetime.now().strftime("%Y-%m-%d")
nome_arquivo_noticias = f'politica_feed_{data_atual}.csv'

In [ ]:
#Criar um modelo de nome de arquivo para base de notícias
caminho_arquivo_noticias = f'{pasta_drive_noticias}{nome_arquivo_noticias}'

In [ ]:
#Criar um modelo de nome de arquivo para o dataframe final criado
pasta_dataframe = '/content/drive/My Drive/PesquisaPoliticos/Noticias/G1/'
nomearquivodataframe = 'Noticiasempilhadas.csv'
caminho_df_final = f'{pasta_dataframe}{nomearquivodataframe}'

In [ ]:
#Obtem dados em xml do link fornecido pelo G1 (Política) e armazena dados das últimas notícias na pasta criada no Google Drive, com o nome do arquivo igual a data de hoje

url = "https://g1.globo.com/dynamo/politica/rss2.xml"

politica = requests.get(url)

soup = BeautifulSoup(politica.content, "xml")
items = soup.find_all("item")

# Verificar se o arquivo já existe (Verificar se já tem notícias do dia atual salvos lá)
if os.path.exists(caminho_arquivo_noticias):
    pass  # Não faz nada se o arquivo já existe
else:  #Se ele existir, faz o código para salvar no Drive
    politica = requests.get(url)
    soup = BeautifulSoup(politica.content, "xml")
    items = soup.find_all("item")

    data = []
    for item in items:
        title = item.find("title").text
        link = item.find("link").text
        pub_date = item.find("pubDate").text
        description = item.find("description").text

        data.append([title, link, pub_date, description])

    # Criar DataFrame com os dados do XML
    df = pd.DataFrame(data, columns=["titulo", "link", "datapublicacao", "descricao"])

    # Salvar o DataFrame como um arquivo CSV
    df.to_csv(caminho_arquivo_noticias, index=False)

Obter os arquivos da pasta do Google drive para um único Dataframe DF

In [ ]:
# Lista para armazenar DataFrames de cada arquivo CSV ou google sheets
dfs = []

# Iterar sobre os arquivos na pasta
for arquivo in os.listdir(pasta_drive_noticias):

        caminho_arquivo = os.path.join(pasta_drive_noticias, arquivo)

        # Ler o arquivo CSV e adicionar ao DataFrame
        df = pd.read_csv(caminho_arquivo)

        # Adicionar o DataFrame à lista
        dfs.append(df)

# Concatenar todos os DataFrames em um único DataFrame
df = pd.concat(dfs, ignore_index=True)

# Remover linhas duplicadas
df = df.drop_duplicates()

In [ ]:
df

,titulo,link,datapublicacao,descricao
0,"Ações penais de 8 de janeiro, revista íntima em presídios, correção do FGTS: o que estará no radar do STF em 2024",https://g1.globo.com/politica/noticia/2024/01/04/acoes-penais-de-8-de-janeiro-revista-intima-em-presidios-correcao-do-fgts-o-que-estara-no-radar-do-stf-em-2024.ghtml,"Thu, 04 Jan 2024 03:00:22 -0000","Corte pode se debruçar sobre uma série de processos com impactos diretos na vida dos cidadãos. Tribunal terá, ainda em fevereiro, a composição completa por conta da posse do ministro Flávio Dino. O Supremo Tribunal Federal (STF) vai começar o ano Judiciário de 2024 com a análise de novos conjuntos de ações penais contra réus pelos atos antidemocráticos de 8 de janeiro. \nA partir de fevereiro, também contará com a composição completa, com a posse de Flávio Dino, prevista para o dia 22. ..."
1,"Governo publicará em fevereiro regras para uso de câmeras corporais por polícias, diz secretário do MJ",https://g1.globo.com/politica/noticia/2024/01/03/governo-publicara-em-fevereiro-regras-para-uso-de-cameras-corporais-por-policias-diz-secretario-do-mj.ghtml,"Wed, 03 Jan 2024 22:53:48 -0000","<img src=""https://s2-g1.glbimg.com/heRysFS_HGKca35oajG0XARSYX0=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2023/K/G/NkIsx1SsyvOE78tROhZg/gnews-cappelli.jpeg"" /><br /> Cappelli citou colaboração das polícias e afirmou que 'ideologizar' segurança pública não faz bem ao Brasil. Em 2022, letalidade policial em São Paulo caiu após uso das câmeras. O secretário-executivo do Ministério da Justiça, Ricardo Cappelli\nAndressa Gonçalves/ GloboNews\nO secretário-..."
2,Brasil condena ataque que deixou quase 100 mortos no Irã: 'Repúdio a todo e qualquer ato de terrorismo',https://g1.globo.com/politica/noticia/2024/01/03/brasil-condena-ataque-que-deixou-quase-100-mortos-no-ira-repudio-a-todo-e-qualquer-ato-de-terrorismo.ghtml,"Wed, 03 Jan 2024 21:38:40 -0000","Explosões mataram 95 pessoas que participavam de procissão com destino ao túmulo de Qassem Soleimani. General iraniano foi morto em 2020 após ataque com drone dos EUA. Explosões no Irã deixam mais de 100 mortos\nO Ministério das Relações Exteriores do Brasil divulgou nota na qual condena as explosões que mataram 95 pessoas que participavam de procissão ao túmulo de Qassem Soleimani nesta quarta-feira (3) em Kerman, no Irã.\nSegundo serviços de emergência do Irã, outras 211 pessoas ficara..."
3,Lupi diz que fila do INSS 'nunca vai acabar' e fala em reduzir espera para 30 dias em 2024,https://g1.globo.com/politica/noticia/2024/01/03/lupi-diz-que-fila-do-inss-nunca-vai-acabar-e-fala-em-reduzir-espera-para-30-dias-em-2024.ghtml,"Wed, 03 Jan 2024 18:28:47 -0000","<img src=""https://s2-g1.glbimg.com/GPzrXm13yU7dB-vYJ7IQSphSlfI=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2023/Y/J/wd8iFzRkSyKCKIGoGNBQ/whatsapp-image-2023-10-18-at-11.24.46.jpeg"" /><br /> Ministro da Previdência deu a declaração em evento com aprovados no último concurso do INSS. Há um ano, quando tomou posse, Lupi disse que pretendia acabar com as filas em 2023. O ministro da Previdência Social, Carlos Lupi, em imagem de 2023\nBasílio Magno/TV Diári..."
4,"Lula reduz número, mas mantém acesso a fuzis para uso pessoal de militares",https://g1.globo.com/politica/blog/octavio-guedes/post/2024/01/03/lula-reduz-numero-mas-mantem-acesso-a-fuzis-para-uso-pessoal-de-militares.ghtml,"Wed, 03 Jan 2024 16:19:21 -0000","<img src=""https://s2-g1.glbimg.com/dtZANw8af_EezcCxlKGqdAcB-jE=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2023/R/g/fKXx6qQoO4tXwD6MkNxw/whatsapp-image-2023-12-15-at-16.49.46.jpeg"" /><br /> Portaria de dezembro do Exército manteve acesso a armas de uso restrito para fardados durante a folga. O presidente Luiz Inácio Lula da Silva (PT)\nFernando Madeira/Rede Gazeta\nA portaria nº 164, editada pelo Exército em dezembro de 2023 com aval do governo Lula (P..."
...,...,...,...,...
2000,Lira não 

Buscar o último arquivo NoticiasEmpilhadasAntigo salvo, que já possui os scores calculados

In [ ]:
noticiasempilhadasantigo = pd.read_csv(caminho_df_final)


In [ ]:
noticiasempilhadasantigo

,titulo,link,datapublicacao,descricao,personalidadescitadas,scorescoluna
0,Bets: veja como votaram os deputados na nova aprovação do PL das apostas esportivas,https://g1.globo.com/politica/noticia/2023/12/22/bets-veja-como-votaram-os-deputados-nova-aprovacao-do-pl-das-apostas-esportivas-que-libera-cassinos-online.ghtml,2023-12-22 11:32:46+00:00,"Pelo texto aprovado, empresas terão que pagar 12% sobre o faturamento, e, apostadores, pagarão imposto de 15% sobre o ganho com as apostas. A matéria recebeu 292 votos favoráveis e 114 contra. A Câmara dos Deputados aprovou nesta quinta-feira (21) o texto do projeto que regulamenta o mercado de apostas esportivas online, as ""bets"". A Câmara e incluiu jogos online, como cassinos, na regulamentação.. (CORREÇÃO: o g1 errou ao publicar, nesta reportagem, que a primeira tabela abaixo listava...","['Márcio Marinho BA Sim', 'Julio Arcoverde PI', 'Murillo Gouvea RJ', 'Luiz Carlos Hauly PR Sim', 'Fausto Pinato SP', 'André Fernandes', 'Marcos Aurélio Sampaio PI', 'Joaquim Passarinho PA Sim', 'Rubens Pereira Júnior MA', 'Mersinho Lucena', 'Filipe Barros PR Não', 'Renilce Nicodemos', 'Tarcísio Motta RJ', 'Túlio Gadêlha', 'Valmir Assunção BA', 'Chris Tonietto', 'Ricardo Ayres TO Sim', 'Julia Zanatta SC Sim', 'Rosângela Reis MG Sim', 'Rodolfo Nogueira MS', 'Zé Haroldo Cathedral RR Não', 'Daia...","[('Abilio Brunini', 'Abilio Brunini', 4.843444347381592), ('Adilson Barroso', 'Adilson Barroso', 4.788721561431885), ('Adolfo Viana BA', 'Adolfo Viana', 4.201511859893799), ('Adriana Ventura', 'Adriana Ventura', 4.911692142486572), ('Adriano do Baldy GO', 'Adriano do Baldy', 4.430189609527588), ('Afonso Hamm RS', 'Afonso Hamm', 4.3113789558410645), ('Airton Faleiro PA', 'Airton Faleiro', 4.286380290985107), ('Alberto Mourão', 'Alberto Mourão', 4.741438388824463), ('Alceu Moreira', 'Alceu Mor..."
1,"Ações penais de 8 de janeiro, revista íntima em presídios, correção do FGTS: o que estará no radar do STF em 2024",https://g1.globo.com/politica/noticia/2024/01/04/acoes-penais-de-8-de-janeiro-revista-intima-em-presidios-correcao-do-fgts-o-que-estara-no-radar-do-stf-em-2024.ghtml,2024-01-04 03:00:22+00:00,"Corte pode se debruçar sobre uma série de processos com impactos diretos na vida dos cidadãos. Tribunal terá, ainda em fevereiro, a composição completa por conta da posse do ministro Flávio Dino. O Supremo Tribunal Federal (STF) vai começar o ano Judiciário de 2024 com a análise de novos conjuntos de ações penais contra réus pelos atos antidemocráticos de 8 de janeiro. . A partir de fevereiro, também contará com a composição completa, com a posse de Flávio Dino, prevista para o dia 22. ...","['Edson Fachin', 'Anderson Gomes', 'Ricardo Lewandowski', 'Marielle', 'Taxa Referencial', 'André Mendonça', 'Luís Roberto Barroso', 'Cristiano Zanin', 'Gilmar Mendes', 'Marielle Franco', 'Anderson', 'Rosa Weber', 'Fornecimento', 'Judiciário', 'Alexandre de Moraes', 'Dias Toffoli', 'Luiz Fux', 'Flávio Dino']",[]
2,"Governo publicará em fevereiro regras para uso de câmeras corporais por polícias, diz secretário do MJ",https://g1.globo.com/politica/noticia/2024/01/03/governo-publicara-em-fevereiro-regras-para-uso-de-cameras-corporais-por-policias-diz-secretario-do-mj.ghtml,2024-01-03 22:53:48+00:00,"Cappelli citou colaboração das polícias e afirmou que 'ideologizar' segurança pública não faz bem ao Brasil. Em 2022, letalidade policial em São Paulo caiu após uso das câmeras. O secretário-executivo do Ministério da Justiça, Ricardo Cappelli. Andressa Gonçalves/ GloboNews. O secretário-executivo do Ministério da Justiça e Segurança Pública, Ricardo Cappelli, disse nesta quarta-feira (3) que o governo publicará em fevereiro as diretrizes nacionais para utilização de câmeras corpora...","['TJ', 'Ricardo Cappelli', 'Ideologizar', 'Capelli', 'Tarcísio', 'Tarcísio de Freitas', 'Cappelli', 'Andressa Gonçalves/ GloboNews']","[('Tarcísio', 'Tarcísio Motta', 4.2149834632873535), ('Cappelli', 'Tadeu Filippelli', 3.821045398712158)]"
3,B

Identificar as linhas contidas no df que não estão  noticiasempilhadasantigo (pela datapublicacao) . Salvar essas linhas em um novo dataframe chamado df_novasnoticias

In [ ]:
# Convertendo a coluna datapublicacao para o formato de data (Necessário para o código seguinte funcionar)
df['datapublicacao'] = pd.to_datetime(df['datapublicacao'])
noticiasempilhadasantigo['datapublicacao'] = pd.to_datetime(noticiasempilhadasantigo['datapublicacao'])

In [ ]:
# Encontrando as linhas em df que não estão em noticiasempilhadasantigo
df_novasnoticias = df[~df['datapublicacao'].isin(noticiasempilhadasantigo['datapublicacao'])]

In [ ]:
df_novasnoticias

,titulo,link,datapublicacao,descricao
1303,Polícia Federal prende mais um homem por suspeita de envolvimento nos assassinatos de Bruno e Dom,https://g1.globo.com/politica/noticia/2024/01/18/dino-diz-que-policia-prendeu-mais-uma-pessoa-por-envolvimento-nos-assassinatos-de-bruno-e-dom.ghtml,2024-01-18 21:32:35+00:00,"<img src=""https://s2-g1.glbimg.com/fRQEQ-MGoLSYNZGUaqO82ob9LEM=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2022/A/i/OYViXNRcOqMQp0EQcwtw/whatsapp-image-2022-06-15-at-20.55.45.jpeg"" /><br /> Foi preso em Tabatinga (AM) Jânio de Freitas de Souza, considerado o braço-direito de Ruben Dario da Silva, acusado de ser o mandante do crime. A Polícia Federal prendeu nesta quinta-feira (18) Jânio Freitas de Souza. Ele é considerado braço-direito de Ruben Dário ..."
1304,"A convite de Nunes, Aldo Rebelo vai assumir Secretaria das Relações Internacionais em SP no lugar de Marta Suplicy",https://g1.globo.com/sp/sao-paulo/noticia/2024/01/18/a-convite-de-nunes-aldo-rebelo-vai-assumir-secretaria-das-relacoes-internacionais-em-sp-no-lugar-de-marta-suplicy.ghtml,2024-01-18 21:23:03+00:00,"<img src=""https://s2-g1.glbimg.com/YG30rwZuwVl0kMR_bKtgz1Yvty0=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2021/z/Y/PCVuDXRfmBaSAvL1pPQA/aldo-rebelo.jpg"" /><br /> Partido de Aldo, o PDT, apoia pré-candidatura de Guilherme Boulos (PSOL) a prefeito de SP. O ex-ministro Aldo Rebelo\nDivulgação\nO ex-deputado federal e ex-ministro Aldo Rebelo (PDT) aceitou o convite do prefeito de São Paulo, Ricardo Nunes (MDB), e vai assumir a Secretaria das Relações Inte..."
1305,"'Vamos criar um Gaeco nacional', diz futuro secretário de Segurança do Ministério da Justiça",https://g1.globo.com/politica/blog/julia-duailibi/post/2024/01/18/entrevista-mario-sarrubbo.ghtml,2024-01-18 21:16:52+00:00,"Sarrubbo é procurador-geral de Justiça de São Paulo e foi convidado pelo futuro ministro da Justiça, Ricardo Lewandowski, para assumir a Secretaria Nacional de Segurança Pública. Mario Sarrubbo fala sobre a criação de um Gaeco nacional\nMário Luiz Sarrubbo, futuro secretário nacional de segurança, disse em entrevista ao GloboNews Mais nesta quinta-feira (18) que será criado um Gaeco nacional para integrar estados e órgãos de Justiça no combate à criminalidade. \n""O que o ministro nos ped..."
1306,"Deputado Carlos Jordy tem arma, celulares, notebook e passaporte apreendidos em operação da PF",https://g1.globo.com/rj/rio-de-janeiro/noticia/2024/01/18/deputado-carlos-jordy-tem-arma-celulares-notebook-e-passaporte-apreendidos-em-operacao-da-pf.ghtml,2024-01-18 20:19:49+00:00,"<img src=""https://s2-g1.glbimg.com/F0ziN_BtO1_ufzaZeRNY6RXSC1c=/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2024/y/X/KfkFsaTcAJnnjhJoEJFg/whatsapp-image-2024-01-18-at-17.00.47.jpeg"" /><br /> Arma e munição estavam legalizadas. Sobre a operação, Jordy declarou que foi vítima de uma ‘pesca probatória’ e que crê ser alvo de uma armação. Pistola e carregadores de Carlos Jordy apreendidos; armamento está legalizado\nReprodução\nAgentes da Polícia Federal apr..."
1307,"Filiação de Marta ao PT será em 2 de fevereiro, diz presidente municipal do partido",https://g1.globo.com/politica/blog/julia-duailibi/post/2024/01/18/filiacao-de-marta-ao-pt-data.ghtml,2024-01-18 17:42:18+00:00,"Decisão foi tomada na noite desta quarta (17); local ainda está em estudo. Filiação de Marta Suplicy ao PT será dia 2 de fevereiro\nO Partido dos Trabalhadores ( PT) bateu o martelo sobre a data do ato de filiação de Marta Suplicy ao partido: será em 2 de fevereiro. A decisão foi tomada na noite desta quarta-feira (16), segundo Laércio Ribeiro, presidente do diretório municipal do partido. \nO evento deve ser à noite, por volta das 18h. O local ainda está em estudo. Como mostrou o blog, ..."
...,...,...,...,...
2000,Lira não consegue encurtar férias dos líderes partidários e desmarca reunião para discutir corte nas emendas,https://g1.glob

In [ ]:
#Alteração do tipo da coluna Data de Publicação para Datetime.
df["datapublicacao"] = pd.to_datetime(df["datapublicacao"])

Análise dos Dados das Notícias do G1

In [ ]:
# Número de linhas e colunas
print("Número de linhas e colunas:", df.shape)

# Informações sobre o DataFrame
print("Informações sobre o DataFrame:")
df_novasnoticias.info()

# Número de valores nulos em cada coluna
print("Número de valores nulos em cada coluna:")
print(df.isnull().sum())



Número de linhas e colunas: (464, 4)
Informações sobre o DataFrame:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 157 entries, 1303 to 2045
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   titulo          157 non-null    object             
 1   link            157 non-null    object             
 2   datapublicacao  157 non-null    datetime64[ns, UTC]
 3   descricao       157 non-null    object             
dtypes: datetime64[ns, UTC](1), object(3)
memory usage: 6.1+ KB
Número de valores nulos em cada coluna:
titulo            0
link              0
datapublicacao    0
descricao         0
dtype: int64


In [ ]:
#Código para saber o intervalo de datas de publicações a ser analisado (tabela com todas as noticias =df)
date_range = df["datapublicacao"].min().strftime("%Y-%m-%d") + " to " + df["datapublicacao"].max().strftime("%Y-%m-%d")
date_range

'2023-12-22 to 2024-01-28'

In [ ]:
#Código para saber o intervalo de datas de publicações a ser analisado (tabelas com as noticias não processadas = df_novasnoticias)
date_range = df_novasnoticias["datapublicacao"].min().strftime("%Y-%m-%d") + " to " + df_novasnoticias["datapublicacao"].max().strftime("%Y-%m-%d")
date_range

'2024-01-18 to 2024-01-28'

In [ ]:
#Contagem do númerod de palavras que avaliaremos na coluna "descricao"
descricao = df['descricao']
all_descricao = " ".join(s for s in descricao)

# ver quantidade de palavras
print("Quantidade de Palavras: {}".format(len(all_descricao)))

Quantidade de Palavras: 1722843


Utilização de REGEX para tirar alguns caracteres indesejáveis


In [ ]:
import re

newline = lambda x: re.sub('\n', '. ', x) # remove \n

df_novasnoticias['descricao'] = df_novasnoticias['descricao'].apply(newline)

<ipython-input-31-fe4c53969edf>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novasnoticias['descricao'] = df_novasnoticias['descricao'].apply(newline)


In [ ]:
removerurl = lambda x: re.sub(re.compile(r'<img.*?/><br />'),' ',x) # remove a url de imagem inicial para limpar o texto
df_novasnoticias['descricao'] = df_novasnoticias['descricao'].apply(removerurl)

<ipython-input-32-5d839dacb7bd>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novasnoticias['descricao'] = df_novasnoticias['descricao'].apply(removerurl)


Aplicação no NER para criação de uma coluna com Entidades = PER (Personalidades)


In [ ]:
#Carregando o modelo em português
nlp = spacy.load("pt_core_news_sm")

In [ ]:
#Extraindo as entidades das palavras/expressões da coluna descricao e salvando em uma lista
personalidadescitadasdescricao = df_novasnoticias['descricao'].\
                    apply(lambda x: [(entidade.text, entidade.label_)\
                                   for entidade in nlp(x).ents if entidade.label_ == 'PER'])

In [ ]:
#Extraindo as entidades das palavras/expressões da coluna titulo e salvando em uma lista
personalidadescitadastitulo = df_novasnoticias['titulo'].\
                    apply(lambda x: [(entidade.text, entidade.label_)\
                                   for entidade in nlp(x).ents if entidade.label_ == 'PER'])

In [ ]:
#Unificando as entidades das colunas descricao e titulo em uma única lista e depois excluir duplicatas
#lista_temp = []
#for item in personalidadescitadasdescricao + personalidadescitadastitulo:
    #if item not in lista_temp:
    #    lista_temp.append(item)


df_novasnoticias['personalidadescitadas']=personalidadescitadastitulo + personalidadescitadasdescricao
df_novasnoticias['personalidadescitadas']=df_novasnoticias['personalidadescitadas'].drop_duplicates()

<ipython-input-36-31f83671dda8>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novasnoticias['personalidadescitadas']=personalidadescitadastitulo + personalidadescitadasdescricao
<ipython-input-36-31f83671dda8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novasnoticias['personalidadescitadas']=df_novasnoticias['personalidadescitadas'].drop_duplicates()


In [ ]:
#Função para extrair somente o primeiro item da tupla (nome da personalidade)
def extracao_primeiro_item(lista):
    if isinstance(lista, list) and lista:
        return [item[0] for item in lista]
    else:
        return None
df_novasnoticias['personalidadescitadas'] = df_novasnoticias['personalidadescitadas'].apply(extracao_primeiro_item)

<ipython-input-37-a73609e6e79a>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novasnoticias['personalidadescitadas'] = df_novasnoticias['personalidadescitadas'].apply(extracao_primeiro_item)


In [ ]:
# Percebi que ainda existiam tuplas duplicadas. Função set tranforma a lista x (personalidades citadas) em um conjunto (e conjunto não permite duplicadas).
# O If é para garantir que a função list(Set(x)) só será aplicada se X for uma lista (ignora os NaNs). Se não for lista, retorna o próprio valor.
df_novasnoticias['personalidadescitadas'] = df_novasnoticias['personalidadescitadas'].apply(lambda x: list(set(x)) if isinstance(x, list) else x)


<ipython-input-38-4e02a76dde9a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novasnoticias['personalidadescitadas'] = df_novasnoticias['personalidadescitadas'].apply(lambda x: list(set(x)) if isinstance(x, list) else x)


Trazer uma lista com o completo dos políticos para os quais quero encontrar citações na coluna "Personalidades Citadas" (FONTE: Dados abertos)

In [ ]:
import requests
import pandas as pd

# URL da qual você deseja obter dados
url = 'http://dadosabertos.camara.leg.br/arquivos/deputados/json/deputados.json'

# Faz a requisição GET
response = requests.get(url)

# Converte a resposta JSON para um DataFrame do pandas
data = response.json()
deputados = data['dados']



# Cria um DataFrame do pandas
df_deps = pd.DataFrame(deputados)

df_deps_filtrado = df_deps.loc[df_deps['idLegislaturaFinal'] >= 56]
nomes = df_deps_filtrado['nome'].tolist()



In [ ]:
quantidade_de_deputados = len(nomes)
print(quantidade_de_deputados)

866


Calcular a similaridade da lista de *deputados nomes_civil* com cada uma das personalidades obtidas na coluna *personalidadescitadas*

In [ ]:
# Importação do modelo BERT, conforme disponível em https://huggingface.co/ruanchaves/bert-base-portuguese-cased-assin2-similarity
model_name = "ruanchaves/bert-base-portuguese-cased-assin2-similarity"

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)
import torch

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/589 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/678k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
#personalidades = df['personalidadescitadas'][0]

In [ ]:
# Função para incluir o score de similaridade com o primeiro nome da lista nomes_civil
# x é a lista de personalidades da coluna 'personalidadescitadas'
# y é a variável primeiro_nome
def funcaosimilaridade_doistextos(x,y):
  model_input = tokenizer(*([x], [y]), padding=True, return_tensors="pt")

  with torch.no_grad():
    output = model(**model_input)
    score = output[0][0].detach().numpy().item()
    return score

In [ ]:
#Lista base de deputados, a qual queremos encontrar similaridade nas notícias

nomes_teste = nomes


In [ ]:
df_novasnoticias

,titulo,link,datapublicacao,descricao,personalidadescitadas
1303,Polícia Federal prende mais um homem por suspeita de envolvimento nos assassinatos de Bruno e Dom,https://g1.globo.com/politica/noticia/2024/01/18/dino-diz-que-policia-prendeu-mais-uma-pessoa-por-envolvimento-nos-assassinatos-de-bruno-e-dom.ghtml,2024-01-18 21:32:35+00:00,"Foi preso em Tabatinga (AM) Jânio de Freitas de Souza, considerado o braço-direito de Ruben Dario da Silva, acusado de ser o mandante do crime. A Polícia Federal prendeu nesta quinta-feira (18) Jânio Freitas de Souza. Ele é considerado braço-direito de Ruben Dário da Silva Villar, conhecido como ""Colômbia"", acusado pelos assassinatos do indigenista Bruno Pereira e do jornalista inglês Dom Phillips.. Bruno e Dom foram mortos em 2022, no Vale do Javari, uma das regiões mais violentas...","[Jânio de Freitas de Souza, Oseney da Costa de Oliveira, Alcir Amaral Teixeira, Bruno Araújo Pereira, Leia, Jânio Freitas de Souza, Jânio, Bruno, Bruno Pereira, Dom Phillips, Ney Bello, Dom, Bello, Jair Bolsonaro, Maxciel Pereira dos Santos, Amarildo da Costa Oliveira, Marcelo Xavier, Jefferson da Silva Lima]"
1304,"A convite de Nunes, Aldo Rebelo vai assumir Secretaria das Relações Internacionais em SP no lugar de Marta Suplicy",https://g1.globo.com/sp/sao-paulo/noticia/2024/01/18/a-convite-de-nunes-aldo-rebelo-vai-assumir-secretaria-das-relacoes-internacionais-em-sp-no-lugar-de-marta-suplicy.ghtml,2024-01-18 21:23:03+00:00,"Partido de Aldo, o PDT, apoia pré-candidatura de Guilherme Boulos (PSOL) a prefeito de SP. O ex-ministro Aldo Rebelo. Divulgação. O ex-deputado federal e ex-ministro Aldo Rebelo (PDT) aceitou o convite do prefeito de São Paulo, Ricardo Nunes (MDB), e vai assumir a Secretaria das Relações Internacionais da capital paulista.. “Recebi convite do prefeito Ricardo Nunes para uma secretaria na cidade de São Paulo. Decidi aceitar. Em comum acordo com o presidente Lupi, pedirei licença do P...","[Nunes, Lupi, Marta Suplicy, Ricardo Nunes, PSOL, Partido de Aldo, Aldo Rebelo, Secretaria das Relações Internacionais, Aldo, Decidi, Guilherme Boulos]"
1305,"'Vamos criar um Gaeco nacional', diz futuro secretário de Segurança do Ministério da Justiça",https://g1.globo.com/politica/blog/julia-duailibi/post/2024/01/18/entrevista-mario-sarrubbo.ghtml,2024-01-18 21:16:52+00:00,"Sarrubbo é procurador-geral de Justiça de São Paulo e foi convidado pelo futuro ministro da Justiça, Ricardo Lewandowski, para assumir a Secretaria Nacional de Segurança Pública. Mario Sarrubbo fala sobre a criação de um Gaeco nacional. Mário Luiz Sarrubbo, futuro secretário nacional de segurança, disse em entrevista ao GloboNews Mais nesta quinta-feira (18) que será criado um Gaeco nacional para integrar estados e órgãos de Justiça no combate à criminalidade. . ""O que o ministro nos ped...","[Mário Luiz Sarrubbo, Mario Sarrubbo, Lewandowski, Ricardo Lewandowski, Senasp, Alexandre de Moraes, Tarcísio de Freitas, Fernando José Martins]"
1306,"Deputado Carlos Jordy tem arma, celulares, notebook e passaporte apreendidos em operação da PF",https://g1.globo.com/rj/rio-de-janeiro/noticia/2024/01/18/deputado-carlos-jordy-tem-arma-celulares-notebook-e-passaporte-apreendidos-em-operacao-da-pf.ghtml,2024-01-18 20:19:49+00:00,"Arma e munição estavam legalizadas. Sobre a operação, Jordy declarou que foi vítima de uma ‘pesca probatória’ e que crê ser alvo de uma armação. Pistola e carregadores de Carlos Jordy apreendidos; armamento está legalizado. Reprodução. Agentes da Polícia Federal apreenderam uma pistola, carregadores, munição, telefones celulares, um notebook, R$ 1 mil em espécie e o passaporte do deputado federal Carlos Jordy (PL), alvo de buscas na 24ª fase da Operação Lesa Pátria, deflagrada nesta...","[Arma, Carlos Jordy, PL, Alexandre de Moraes, Jordy, Deputado Carlos Jordy, PL-RJ, Fui]"
1307,"Filiação de Marta ao PT será em 2 de fevereiro, diz presidente municipal do partido",https://g1.globo.com/politica/blog/julia-duailibi/post/202

In [ ]:
df_novasnoticias = df_novasnoticias.head(5)

In [ ]:

df_novasnoticias_copy = df_novasnoticias.copy()

# Função para aplicar a similaridade a cada linha
def calcular_similaridade(row):
    resultados = []

    if row['personalidadescitadas'] is not None:  # Verificar se a lista não é None
        for i, x in enumerate(row['personalidadescitadas']):
            for j, deputado in enumerate(nomes_teste):
                resultado = (x, deputado, funcaosimilaridade_doistextos(x, deputado))
                if resultado[2] >= 3.8:  # Check if similarity score is >= 3.8
                    resultados.append(resultado)

                # Mensagem de impressão para acompanhar o andamento
                print(f"Progresso: Linha {row.name}, Iteração deputado {j+1}/{len(nomes_teste)}, Iteração texto {i+1}/{len(row['personalidadescitadas'])} concluída.")

    return resultados

# Aplicar a função a cada linha e armazenar os resultados na nova coluna 'scorescoluna'
df_novasnoticias['scorescoluna'] = df_novasnoticias_copy.apply(calcular_similaridade, axis=1)


A saída de streaming foi truncada nas últimas 5000 linhas.
Progresso: Linha 1307, Iteração deputado 197/866, Iteração texto 8/13 concluída.
Progresso: Linha 1307, Iteração deputado 198/866, Iteração texto 8/13 concluída.
Progresso: Linha 1307, Iteração deputado 199/866, Iteração texto 8/13 concluída.
Progresso: Linha 1307, Iteração deputado 200/866, Iteração texto 8/13 concluída.
Progresso: Linha 1307, Iteração deputado 201/866, Iteração texto 8/13 concluída.
Progresso: Linha 1307, Iteração deputado 202/866, Iteração texto 8/13 concluída.
Progresso: Linha 1307, Iteração deputado 203/866, Iteração texto 8/13 concluída.
Progresso: Linha 1307, Iteração deputado 204/866, Iteração texto 8/13 concluída.
Progresso: Linha 1307, Iteração deputado 205/866, Iteração texto 8/13 concluída.
Progresso: Linha 1307, Iteração deputado 206/866, Iteração texto 8/13 concluída.
Progresso: Linha 1307, Iteração deputado 207/866, Iteração texto 8/13 concluída.
Progresso: Linha 1307, Iteração deputado 208/866, 

<ipython-input-48-4f00ac0278fa>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novasnoticias['scorescoluna'] = df_novasnoticias_copy.apply(calcular_similaridade, axis=1)


In [ ]:
# resultados = []

# for deputado in nomes_teste:
#     # Cria uma cópia da tabela para cada deputado
#     df_novasnoticias_copy = df_novasnoticias.copy()

#     # Adiciona a coluna 'scorescoluna' para o deputado atual
#     df_novasnoticias_copy.loc[:, 'scorescoluna'] = df_novasnoticias_copy['personalidadescitadas'].apply(
#         lambda personalidades: sorted([(x, deputado, funcaosimilaridade_doistextos(x, deputado)) for x in personalidades if funcaosimilaridade_doistextos(x, deputado) >= 4], key=lambda x: x[1], reverse=True) if personalidades else [])

#     # Adiciona a cópia da tabela ao resultado
#     resultados.append(df_novasnoticias_copy)

# # Concatena todos os resultados em um único DataFrame
# resultado_final = pd.concat(resultados, ignore_index=True)
#   #df_filtro_deputado = df[df['scorescoluna'].apply(condicao_filtro)]
#   #nome_arquivo = f'{pasta_drive_noticias_deputados}noticias_{deputado}.csv'
#   #df_filtro_deputado.to_csv(nome_arquivo, index=False)
#   #if funcaosimilaridade_doistextos(x, deputado) >= 4


In [ ]:
df_final = pd.concat([noticiasempilhadasantigo, df_novasnoticias], ignore_index=True)

In [ ]:
#Gravação do df NotíciasEmpilhadas completo no Drive (Já com os scores)
df_final.to_csv(caminho_df_final, index=False, mode='w')